# LibMagic Setup Helper Functions
This notebook contains utility functions to help install and set up libmagic based on the operating system.

In [ ]:
import os
import platform
import subprocess
import sys
import shutil
from pathlib import Path

def detect_os():
    """
    Detect the operating system.

    Returns:
        str: One of 'windows', 'macos', 'linux', or 'unknown'
    """
    system = platform.system().lower()
    if system == 'darwin':
        return 'macos'
    elif system == 'linux':
        return 'linux'
    elif system == 'windows':
        return 'windows'
    else:
        return 'unknown'

def detect_linux_distro():
    """
    Detect the Linux distribution if running on Linux.

    Returns:
        str: Linux distribution name or 'unknown'
    """
    try:
        if os.path.exists('/etc/os-release'):
            with open('/etc/os-release', 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.startswith('ID='):
                        return line.split('=')[1].strip().strip('"')

        # Try commands for older distributions
        if shutil.which('lsb_release'):
            return subprocess.check_output(['lsb_release', '-si'],
                                          universal_newlines=True).strip().lower()
    except Exception:
        pass

    return 'unknown'

def install_libmagic_macos():
    """
    Install libmagic on macOS using Homebrew.

    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Check if Homebrew is installed
        if not shutil.which('brew'):
            print("Homebrew not found. Installing Homebrew...")
            subprocess.check_call('/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"',
                                 shell=True)

        # Install libmagic
        subprocess.check_call(['brew', 'install', 'libmagic'])
        print("libmagic installed successfully on macOS")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error installing libmagic on macOS: {e}")
        return False

def install_libmagic_linux(distro):
    """
    Install libmagic on Linux.

    Args:
        distro (str): Linux distribution name

    Returns:
        bool: True if successful, False otherwise
    """
    try:
        if distro in ['ubuntu', 'debian', 'linuxmint', 'pop']:
            subprocess.check_call(['sudo', 'apt-get', 'update'])
            subprocess.check_call(['sudo', 'apt-get', 'install', '-y', 'libmagic-dev'])
        elif distro in ['fedora', 'rhel', 'centos']:
            subprocess.check_call(['sudo', 'dnf', 'install', '-y', 'file-devel'])
        elif distro in ['arch', 'manjaro']:
            subprocess.check_call(['sudo', 'pacman', '-S', '--noconfirm', 'file'])
        elif distro in ['opensuse', 'suse']:
            subprocess.check_call(['sudo', 'zypper', 'install', '-y', 'file-devel'])
        else:
            print(f"Unsupported Linux distribution: {distro}")
            print("Please install libmagic manually.")
            return False

        print(f"libmagic installed successfully on {distro}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error installing libmagic on Linux: {e}")
        return False

def install_libmagic_windows():
    """
    Install libmagic on Windows.

    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Check if pip is available
        if not shutil.which('pip'):
            print("pip not found. Please install pip.")
            return False

        # On Windows, we rely on python-magic-bin which includes the necessary DLLs
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-magic-bin'])
        print("libmagic (python-magic-bin) installed successfully on Windows")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error installing libmagic on Windows: {e}")
        return False

def check_libmagic_installed():
    """
    Check if libmagic is properly installed.

    Returns:
        bool: True if installed, False otherwise
    """
    try:
        import magic
        # Try to use the library with a small test
        test_file = Path(__file__).resolve()
        magic.from_file(str(test_file))
        return True
    except ImportError:
        print("python-magic is not installed. Installing...")
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-magic'])
            return check_libmagic_installed()  # Recursive call to verify
        except subprocess.CalledProcessError:
            return False
    except Exception as e:
        print(f"libmagic is installed but not working correctly: {e}")
        return False

def setup_libmagic():
    """
    Main function to ensure libmagic is properly installed.

    Returns:
        bool: True if setup was successful, False otherwise
    """
    # First check if libmagic is already working
    if check_libmagic_installed():
        print("lib